### Install liten
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ liten
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/liten
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/liten/dist/liten-0.0.1-py3-none-any.whl
```

Import Apache arrow

In [ ]:
import pyarrow as pa
from pyarrow import csv

Import Liten-ten is local rten is remote. rten imports pyarrow library as well.

In [ ]:
import liten as ten

In [ ]:
import liten.rcliten as rten

Import Ray to be used as a cluster

In [ ]:
import ray

Start a cluster with single worker.

In [ ]:
ray.init(num_cpus=1)

In [ ]:
ray.cluster_resources()

Create a Liten Cache Actor. It is residing on a remote node, and being executed on that node. tc is the Liten Cache actor handle.

In [ ]:
rten.RCLiten = ray.remote(rten.RCLiten)
tc = rten.RCLiten.remote()

These are fact and dimension tables of TPCH. Read them remotely.

In [ ]:
fact_tables = ['lineitem']
dim_tables = ['customer','orders','supplier','nation','region']
tpch_dir = '/mnt/c/Users/hkver/Documents/dbai/tpch-kit/sf1g/'

In [ ]:
def read_tables(tables, table_type):
    arrow_tables = []
    for table_name in tables:
        tpch_table = tpch_dir+table_name+'.tbl'
        print('Reading ', tpch_table)
        tc.set_table.remote(table_name, table_type)
        pytable = tc.read_csv.remote(input_file=tpch_table, parse_options=csv_options)
        # print(' Rows=', pytable.num_rows,' Cols=', pytable.num_columns)
        arrow_tables.append(pytable)
    return arrow_tables

In [ ]:
%%time
csv_options = pa.csv.ParseOptions(delimiter='|')
pa_fact_tables = read_tables(fact_tables, 1)
pa_dim_tables = read_tables(dim_tables, 0)

In [ ]:
tc.info.remote()

Read a table into TCache

In [ ]:
%%time
result = tc.make_dtensor.remote()

Read Arrow table

In [ ]:
result = tc.query6.remote()

In [ ]:
result = tc.query5.remote()

This will kill remote Liten Cache.

In [ ]:
ray.kill(tc)

Shut down ray now

In [ ]:
ray.shutdown()